# AI Agent Examples

In this chapter, you will explore AI agents that can decide which tools to use and remember context across turns.

**Workflow files:** The JSON files for this chapter are in `courses/n8n_no_code/workflows/`

| File | What it demonstrates |
|------|---------------------|
| `05_ai_agent_basics_calculator_memory.json` | Agent with Calculator tool + memory |
| `06_ai_agent_tools_wikipedia_calculator.json` | Agent with Wikipedia + Calculator (no extra API keys) |
| `07_ai_agent_chat_trigger_memory.json` | Chat interface with memory across turns |

**Prerequisites:**
- n8n running locally
- An API key for a chat model provider (OpenAI, Anthropic, OpenRouter, etc.)
- Optional: SerpAPI key if you want live web search (the Wikipedia tool needs no extra credentials)

---

## What You Will Build

A question-answering agent that:

1. Receives a user question
2. Decides if it needs to use a tool (search, calculate, look up)
3. Calls the tool and reads results
4. Provides a summarized answer

**Example interaction:**

- **Input:** "What is the square root of 144?"
- **Agent thinks:** "I need to calculate this" → calls Calculator tool
- **Output:** "The square root of 144 is 12."

---

## Step 1: Create the Workflow

1. Open n8n at `http://localhost:5678`
2. Click **Add Workflow**
3. Name it "My First AI Agent"

---

## Step 2: Add the Trigger

1. Click the **+** button to add a node
2. Search for **Manual Trigger**
3. Add it to the canvas

This lets you test the workflow by clicking "Execute Workflow."

---

## Step 3: Add Input Data

1. Click **+** after Manual Trigger
2. Search for **Edit Fields** (or Set)
3. Configure it:
   - Click **Add Field**
   - Name: `question`
   - Value: `What are the latest developments in AI regulation?`
   - Type: String

This simulates a user asking a question.

---

## Step 4: Add the AI Agent Node

1. Click **+** after Edit Fields
2. Search for **AI Agent**
3. Add it to the canvas
4. Configure:
   - **Agent Type:** Tools Agent
   - **Text (Human Message):** Click the expression toggle (gears icon) and enter:
     ```
     {{ $json.question }}
     ```
   - **System Message:**
     ```
     You are a helpful research assistant. When asked a question:
     1. If you need current information, use the search tool
     2. Summarize your findings clearly
     3. Cite that you searched the web if you did
     ```

---

## Step 5: Connect a Chat Model

The AI Agent needs a "brain" to think with.

1. Click the **AI Agent** node to select it
2. Look for the **Model** sub-node connector (dotted circle)
3. Click it and search for your preferred Chat Model (e.g., **OpenAI Chat Model**, **Anthropic Chat Model**, or **OpenRouter Chat Model**)
4. Configure:
   - **Credential:** Select or create your API credential
   - **Model:** Choose any supported model from your provider

You should see a **dotted line** connecting the Chat Model to the Agent.

---

## Step 6: Add Tools

Give the agent abilities by connecting tool sub-nodes.

**No-credential tools (recommended for learning):**
- **Calculator** — deterministic math operations
- **Wikipedia** — look up factual information

**Credential-required tools:**
- **SerpAPI** — live web search (requires API key)
- **HTTP Request Tool** — call any API

To add a tool:
1. Click the **AI Agent** node
2. Look for the **Tool** sub-node connector (dotted circle)
3. Click it and search for **Calculator** or **Wikipedia**
4. Add as many tools as needed

You should see **dotted lines** connecting each tool to the Agent.

---

## Step 7: Test the Agent

1. Click **Execute Workflow** (play button)
2. Watch the execution:
   - Manual Trigger runs
   - Edit Fields creates the question
   - AI Agent receives the question
   - Agent decides to search (you may see multiple tool calls)
   - Agent outputs final answer

3. Click the **AI Agent** node to see the output

**Expected output:**
```json
{
  "output": "Based on my search, here are the latest developments in AI regulation..."
}
```

---

## Understanding Agent Behavior

### How the Agent Decides

The agent follows this loop:

1. **Read** the question and system message
2. **Think:** "Do I have enough information to answer?"
3. **If no:** Call a tool (search, calculate, etc.)
4. **Read** the tool's response
5. **Think:** "Now do I have enough?"
6. **Repeat** until confident, then output the answer

### Viewing Agent Reasoning

To see what the agent is thinking:

1. Click the AI Agent node after execution
2. Look at the **Output** panel
3. Expand the execution details to see:
   - Which tools were called
   - What data each tool returned
   - The agent's intermediate reasoning

---

## Common Issues and Fixes

### Agent Keeps Calling Tools Forever

**Cause:** The system message is too vague.

**Fix:** Be specific about when to stop:
```
Search once for current information, then provide your answer.
Do not search more than twice.
```

### Agent Never Uses Tools

**Cause:** The question does not require external data, or the system message discourages tool use.

**Fix:** Ask questions that clearly need current information:
```
What is the weather in Tokyo right now?
```

### Credential Errors

**Cause:** API key not configured or expired.

**Fix:** Go to **Settings → Credentials** and verify your API keys are correct.

---

## Extending Your Agent

Once the basic agent works, try these enhancements:

### Add Memory

Connect a **Simple Memory** sub-node so the agent remembers previous interactions. This enables multi-turn conversations where the agent can reference earlier context.

To add memory:
1. Click the AI Agent node
2. Find the **Memory** sub-node connector
3. Add **Simple Memory** (or Window Buffer Memory)
4. Set a unique **Session ID** if you want separate conversation threads

### Use Chat Trigger for Multi-Turn Conversations

**Recommended for memory demos:** Replace Manual Trigger with **Chat Trigger** to get a proper chat interface. See workflow `07_ai_agent_chat_trigger_memory.json`.

Note: Chat Trigger must connect directly to an agent or chain root node.

### Add More Tools

Connect additional tools so the agent can:
- **Wikipedia:** Look up factual information
- **Calculator:** Perform math operations
- **HTTP Request:** Call specific APIs
- **Code Tool:** Run custom JavaScript logic

---

## Summary

You built an AI agent that:

1. Receives a question from user input
2. Decides if it needs to search the web
3. Calls the search tool when needed
4. Summarizes findings into a response

**Key concepts:**

| Concept | What You Learned |
|---------|------------------|
| **AI Agent node** | Orchestrates tool calls in a loop |
| **Chat Model** | Provides reasoning capability (dotted line) |
| **Tools** | Actions the agent can take (dotted line) |
| **System Message** | Controls agent behavior and boundaries |

**Next steps:** Experiment with different tools and system messages to build agents for your specific use cases.

**Docs:** [AI Agent Node](https://docs.n8n.io/integrations/builtin/cluster-nodes/root-nodes/n8n-nodes-langchain.agent/)